# Boolean logic

## Sources

* Nisan and Schocken, *The Elements of Computing Systems: Building a Modern Computer from First Principles* (2008), Chapter 1: Boolean Logic
* Mims, *Workbook I: Basic Electronics, Transistors and Integrated Circuits*, second printing (2013)
* Mims, *Workbook II: Digital Logic Projects*, second printing (2013)

## 1.1.1 Boolean Algebra

### Boolean expressions

$ f(x, y, z) = (x + y) \cdot \overline{z} $

`(x OR y) AND NOT z`

### Canonical representation

$ f(x, y, z) = \overline{x}y\overline{z} + x\overline{y}\overline{z} + xy\overline{z} $

`((NOT x) AND y AND (NOT z)) OR (x AND (NOT y) AND (NOT z)) OR (x AND y AND (NOT z))`

### Truth table representation

In [2]:
def print_truth_table(f, arr):
    for row in arr:
        print(row, f(*row))

Evaluate using bitwise operators:

In [2]:
def foo(x, y, z):
    return (~x & y & ~z) | (x & ~y & ~z) | (x & y & ~z);

vals = [ [0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1],
         [1, 0, 0], [1, 0, 1], [1, 1, 0], [1, 1, 1] ]

print_truth_table(foo, vals)

[0, 0, 0] 0
[0, 0, 1] 0
[0, 1, 0] 1
[0, 1, 1] 0
[1, 0, 0] 1
[1, 0, 1] 0
[1, 1, 0] 1
[1, 1, 1] 0



## 1.2.1 The Nand Gate

$\overline{x \cdot y}$

`NOT (x AND y)`

### Truth table representation: direct

`if a = b = 1 then out = 0 else out = 1`

In [4]:
def _nand(a, b):
    if a == b == 1:
        return 0
    else:
        return 1

vals = [ [0, 0], [0, 1], [1, 0], [1, 1] ]
print_truth_table(_nand, vals)

[0, 0] 1
[0, 1] 1
[1, 0] 1
[1, 1] 0


## 1.2.3 Basic Logic Gates

(See also https://en.wikipedia.org/wiki/NAND_logic)

### Summary

`Not` built with `Nand` (`_n_` prefix refers to `Nand`):
```py
def _n_not(a):
    return _nand(a, a)
```

`And` built with `Nand` and `Not`:
```py
def _n_and(a, b):
    return _n_not(_nand(a, b))
```

`Or` built with `Nand`, `Not`, and `And`:
```py
def _n_or(a, b):
    return _n_not(_n_and(_nand(a, a), _nand(b, b)))
```

### Basic Logic Gates: `Not`

$\overline{x}$

`NOT (x)`

Converts its input from 0 to 1 and from 1 to 0.

#### Truth table representation: direct

`if in = 0 then out = 1 else out = 0`

In [5]:
def _not(a):
    if a == 0:
        return 1
    else:
        return 0

vals = [ [0], [1] ]
print_truth_table(_not, vals)

[0] 1
[1] 0


#### Truth table representation: Nand logic

Joining the inputs of a Nand gate leaves only the NOT gate:

`Not(a) = Nand(a, a)`

In [5]:
def _n_not(a):
    return _nand(a, a)

vals = [ [0], [1] ]
print_truth_table(_n_not, vals)

[0] 1
[1] 0


#### In HDL (using built-in primitive `Nand` chip)

```verilog
/**
 * Not gate:
 * out = not in
 */

CHIP Not {
    IN in;
    OUT out;

    PARTS:
    
    // Using built-in primitive Nand chip:
    // Not(a) = Nand(a, a)
    Nand(a = in, b = in, out = out);
}
```


### Basic Logic Gates: `And`

$ x \cdot y $

`x AND y`

Returns 1 when both its inputs are 1, else 0.

#### Truth table representation: direct

`if a = b = 1 then out = 1 else out = 0`

In [7]:
def _and(a, b):
    if a == b == 1:
        return 1
    else:
        return 0

vals = [ [0, 0], [0, 1], [1, 0], [1, 1] ]
print_truth_table(_and, vals)

[0, 0] 0
[0, 1] 0
[1, 0] 0
[1, 1] 1


#### Circuit representation

Two switches connected in series. The LED will only be illuminated if both switches are closed.

In [7]:
display(HTML("<iframe frameborder='0' height='448' marginheight='0' marginwidth='0' scrolling='no' \
src='https://123d.circuits.io/circuits/1836053-and-gate-using-two-switches-and-led/embed#breadboard' \
width='650'></iframe>"))

#### Truth table representation: Nand logic

`AND = NOT NAND`

`And(a, b) = Not(Nand(a, b))`

In [6]:
def _n_and(a, b):
    return _n_not(_nand(a, b))

vals = [ [0, 0], [0, 1], [1, 0], [1, 1] ]
print_truth_table(_n_and, vals)

[0, 0] 0
[0, 1] 0
[1, 0] 0
[1, 1] 1


#### In HDL

```verilog
/**
 * And gate:
 * out = 1 if (a == 1 and b == 1)
 *       0 otherwise
 */

CHIP And {
    IN a, b;
    OUT out;

    PARTS:

    // Using built-in primitive Nand chip
    // and previously built Not chip:
    // And(a, b) = Not(Nand(a, b))
    Nand(a = a, b = b, out = c);
    Not(in = c, out = out);
}
```

### Basic Logic Gates: `Or`

$ x + y $

`x OR y`

Returns 1 when at least one of its inputs is 1, and 0 otherwise.

#### Truth table representation: direct

`if a = b = 0 then out = 0 else out = 1`

In [3]:
def _or(a, b):
    if a == b == 0:
        return 0
    else:
        return 1

vals = [ [0, 0], [0, 1], [1, 0], [1, 1] ]
print_truth_table(_or, vals)

[0, 0] 0
[0, 1] 1
[1, 0] 1
[1, 1] 1


#### Truth table representation: Nand logic

`OR = NOT (NAND AND NAND)`

`Or(a, b) = Not(And(Nand(a, a), Nand(b, b)))`

In [7]:
def _n_or(a, b):
    return _n_not(_n_and(_nand(a, a), _nand(b, b)))

vals = [ [0, 0], [0, 1], [1, 0], [1, 1] ]
print_truth_table(_n_or, vals)

[0, 0] 0
[0, 1] 1
[1, 0] 1
[1, 1] 1


#### In HDL

```verilog
 /**
 * Or gate:
 * out = 1 if (a == 1 or b == 1)
 *       0 otherwise
 */

CHIP Or {
    IN a, b;
    OUT out;

    PARTS:

    // Using built-in primitive Nand chip
    // and previously built Not and And chips:
    // Or(a, b) = Not(And(Nand(a, a), Nand(b, b)))
    Nand(a = a, b = a, out = c);
    Nand(a = b, b = b, out = d);
    And(a = c, b = d, out = e);
    Not(in = e, out = out);
}

```

### Basic Logic Gates: `Xor`

WORKINGHERE

In [19]:
from IPython.core.display import display, HTML
styles = open("dark.css", "r").read()
HTML(styles)